# EXTRACT EVENTS EMS

In [2]:
import feedparser as fp
import pandas as pd
import ssl

An error may be raised when running `fp.parse`. I was able to solve changing the module function as explained here: https://github.com/kurtmckee/feedparser/pull/131

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
feeds=fp.parse('https://emergency.copernicus.eu/mapping/activations-rapid/feed')

In [4]:
feeds

{'feed': {'language': 'en',
  'title': 'Copernicus EMS Rapid Mapping Activations',
  'title_detail': {'type': 'text/plain',
   'language': 'en',
   'base': 'https://emergency.copernicus.eu/mapping/activations-rapid/feed',
   'value': 'Copernicus EMS Rapid Mapping Activations'},
  'subtitle': 'This feed contains latest Copernicus EMS - Rapid Mapping activations.',
  'subtitle_detail': {'type': 'text/html',
   'language': 'en',
   'base': 'https://emergency.copernicus.eu/mapping/activations-rapid/feed',
   'value': 'This feed contains latest Copernicus EMS - Rapid Mapping activations.'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://emergency.copernicus.eu/mapping/'},
   {'rel': 'self',
    'href': 'https://emergency.copernicus.eu/mapping/activations-rapid/feed',
    'type': 'application/atom+xml'}],
  'link': 'https://emergency.copernicus.eu/mapping/',
  'tags': [{'term': 'copernicus', 'scheme': None, 'label': None},
   {'term': 'ems', 'scheme': None, 'l

In [5]:
print(len(feeds['entries']))

379


In [6]:
from bs4 import BeautifulSoup
print(feeds['entries'][0]['summary'])

<br />
				<b>Type of Event:</b> Flood (Riverine flood);<br />
				<b>Date/Time of Event (UTC):</b> 2020-01-13 00:00:00;<br />
				<b>Activation Date/Time (UTC):</b> 2020-01-13 17:15:00;<br />
				<b>Dedicated Mechanism:</b> Copernicus EMS (RM);<br />
				<b>Affected Country:</b> Iran;<br />
				<b>Affected Area:</b> ;<br />
				<b>Event Description:</b><br /><p>Days of heavy rain from 09 January 2020 caused severe flooding in southern Iran in Sistan and Baluchestan, Hormozgan and Kerman provinces.<br />
Flooding has affected wide areas of Sistan and Baluchestan, including the capital Zahedan, as well as Konarak, Saravan, Nik Shahr, Delgan, Bazman, Chabahar, Zarābād and Khash. Flooding has blocked roads and damaged houses, bridges, crops and infrastructure. Dozens of areas have been left isolated after roads were damaged or blocked. Schools have been closed in some affected areas.</p>
<br />
				<br />
			<b>Details, maps:</b><br />&#9656;  <a target="_blank" title="[EMSR419] Flood in 

In [7]:
list_of_events=list()
for entry in feeds['entries']:
    #print(entry.keys())
    #print(entry['summary_detail']['value'])
    #we loop throught the whole list of activations
    
    #soup munch data in html format
    soup = BeautifulSoup(entry['summary_detail']['value'], 'html.parser')
    #some sanitation to get the 1st 6 fields
    data=soup.prettify().replace('<br/>','').split('<p>')
    #replace('\n','').replace('<div>','').replace('</div>','').split('<br/>')
    first_half=data[0].replace('<b>','').replace('</b>','').split(';')
    first_half_splitted=[first_half_sp.replace('\n','').lstrip() for first_half_sp in first_half]
    activation=dict()
    for item in first_half_splitted[:-1]:
        kv=item.split(':')
        #need a check because sometimes ; is added to locations
        if len(kv)<2 :
            pass
        else:
            k=kv[0]
            v=kv[1]
            activation[k]=v
        
        
    #now second part with link and imgs 
    #print('<p>'+data[1])
    soup=BeautifulSoup('<p>'+data[1], 'html.parser')
    #links contains details and the feed url
    links=soup.find_all('a',href=True)
    #img is the marker for maps
    marker=soup.find_all('img')
    activation['Event Description']=soup.find('p').get_text().replace('\n','')
    if len(links)>0: 
        activation['Details, maps:']=links[0]['href']
        activation['Activation Feed (GeoRSS)']=links[1]['href']
        activation['Activation Location (preview)']=marker[0]['src']

    else:
        activation['Details, maps:']=''
        activation['Activation Feed (GeoRSS)']=''
        activation['Activation Location (preview)']=''
    
    if 'flood' in activation['Type of Event'].lower(): list_of_events.append(activation)
print(list_of_events)

[{'Type of Event': 'Flood (Riverine flood)', 'Date/Time of Event (UTC)': '2020-01-13 00', 'Activation Date/Time (UTC)': '2020-01-13 17', 'Dedicated Mechanism': 'Copernicus EMS (RM)', 'Affected Country': 'Iran', 'Affected Area': '', 'Event Description': ' Days of heavy rain from 09 January 2020 caused severe flooding in southern Iran in Sistan and Baluchestan, Hormozgan and Kerman provinces.  Flooding has affected wide areas of Sistan and Baluchestan, including the capital Zahedan, as well as Konarak, Saravan, Nik Shahr, Delgan, Bazman, Chabahar, Zarābād and Khash. Flooding has blocked roads and damaged houses, bridges, crops and infrastructure. Dozens of areas have been left isolated after roads were damaged or blocked. Schools have been closed in some affected areas.', 'Details, maps:': 'https://emergency.copernicus.eu/mapping/list-of-components/EMSR419', 'Activation Feed (GeoRSS)': 'https://emergency.copernicus.eu/mapping/list-of-components/EMSR419/feed', 'Activation Location (previe

In [8]:
type(list_of_events)

list

In [9]:
data = pd.DataFrame.from_records(list_of_events)
data

,Type of Event,Date/Time of Event (UTC),Activation Date/Time (UTC),Dedicated Mechanism,Affected Country,Affected Area,Event Description,"Details, maps:",Activation Feed (GeoRSS),Activation Location (preview)
0,Flood (Riverine flood),2020-01-13 00,2020-01-13 17,Copernicus EMS (RM),Iran,,Days of heavy rain from 09 January 2020 cause...,https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...
1,Flood (Riverine flood),2019-12-19 01,2019-12-23 14,Copernicus EMS (RM),Portugal,,"Starting from 19/12/2019, severe storms affec...",https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...
2,Flood (Flash flood),2019-12-13 20,2019-12-13 20,Copernicus EMS (RM),France,,Another round of bad weather swept France’s A...,https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...
3,Flood (Riverine flood),2019-11-19 00,2019-11-27 12,Copernicus EMS (RM),Austria,,"In mid-November 2019, numerous landslides and...",https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...
4,Flood (Flash flood),2019-11-25 03,2019-11-26 12,Copernicus EMS (RM),Greece,,Heavy rainfall occurred early Monday (25/11/2...,https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...
...,...,...,...,...,...,...,...,...,...,...
130,Flood,2012-11-05 07,2012-11-06 11,Copernicus EMS (RM),Slovenia,East and north-east of the Country,Due to heavy rainfall in the whole territory ...,https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...
131,Flood,2012-09-06 11,2012-09-07 15,Copernicus EMS (RM),Cameroon,North and Extreme North,"International Charter call, for which JRC has...",http://www.disasterscharter.org/web/charter/ac...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...
132,Flood,2012-08-24 11,2012-08-28 16,Copernicus EMS (RM),Niger,"Niamey, Dosso, Agades",Floods in Niger have caused several victims.,https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...
133,Flood,2012-08-06 17,2012-08-14 21,Copernicus EMS (RM),Philippines,"Olongapo, Malolos, Manila, Pampanga",Torrential rains brought the Philippines capi...,https://emergency.copernicus.eu/mapping/list-o...,https://emergency.copernicus.eu/mapping/list-o...,http://maps.googleapis.com/maps/api/staticmap?...


In [10]:
data.rename(columns={'Activation Feed (GeoRSS)': 'id_EMS', 'Affected Area': 'area', 'Affected Country': 'countries', 'Date/Time of Event (UTC)': 'start_date', 'Type of Event': 'type'}, inplace=True)
data = data.drop(columns=['Activation Location (preview)', 'Dedicated Mechanism','Details, maps:', 'Event Description', 'Activation Date/Time (UTC)'])
data

,type,start_date,countries,area,id_EMS
0,Flood (Riverine flood),2020-01-13 00,Iran,,https://emergency.copernicus.eu/mapping/list-o...
1,Flood (Riverine flood),2019-12-19 01,Portugal,,https://emergency.copernicus.eu/mapping/list-o...
2,Flood (Flash flood),2019-12-13 20,France,,https://emergency.copernicus.eu/mapping/list-o...
3,Flood (Riverine flood),2019-11-19 00,Austria,,https://emergency.copernicus.eu/mapping/list-o...
4,Flood (Flash flood),2019-11-25 03,Greece,,https://emergency.copernicus.eu/mapping/list-o...
...,...,...,...,...,...
130,Flood,2012-11-05 07,Slovenia,East and north-east of the Country,https://emergency.copernicus.eu/mapping/list-o...
131,Flood,2012-09-06 11,Cameroon,North and Extreme North,https://emergency.copernicus.eu/mapping/list-o...
132,Flood,2012-08-24 11,Niger,"Niamey, Dosso, Agades",https://emergency.copernicus.eu/mapping/list-o...
133,Flood,2012-08-06 17,Philippines,"Olongapo, Malolos, Manila, Pampanga",https://emergency.copernicus.eu/mapping/list-o...


In [11]:
data['start_date'] = pd.to_datetime(data['start_date'], errors='coerce')

In [12]:
data = data.sort_values(by='start_date')
data.index = data.pop('start_date')

In [13]:
data = data.loc['2018-08-01':'2019-04-01']

In [14]:
data.index = data.index.normalize()

In [15]:
data['in_EMS'] = 1

In [16]:
data.head()

,type,countries,area,id_EMS,in_EMS
start_date,,,,,
2018-09-02,Flood (Flash flood),Germany,,https://emergency.copernicus.eu/mapping/list-o...,1
2018-09-18,Flood (Riverine flood),Nigeria,,https://emergency.copernicus.eu/mapping/list-o...,1
2018-09-22,Flood (Flash flood),Tunisia,,https://emergency.copernicus.eu/mapping/list-o...,1
2018-10-06,Flood (Flash flood),"Honduras, Nicaragua",,https://emergency.copernicus.eu/mapping/list-o...,1
2018-10-10,Flood (Flash flood),Spain,,https://emergency.copernicus.eu/mapping/list-o...,1


In [17]:
outfile = "./data/EMS/database.csv"
data.to_csv(outfile, sep=',')